## WEEK-1 DETAILS:

### Methodology used:

 In this project, I have focused on the corona virus cases in Indian States
 I have made a scrapper to collect data of all the states from two sources:
#### 1) Wikipedia corona virus in india page
       https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_India
       From this website it collects the data on total cases,actice cases,deaths & recovered
#### 2) Wikipedia Indian States and UTs population page
       https://en.wikipedia.org/wiki/List_of_states_and_union_territories_of_India_by_population
       From this website it collects the data on the population in each state,population density,
       part of the population which lives in rural and urban areas of the state

### Approach:
    1) Scrapping the data from source 1 into df1
    2) Scrapping the data from source 2 into df2
    3) Sorting df1 and df2 by state name
    4) Concatnating both the dataframes to make a final df
    5) Adding one more column which would show the ratio of Urban population to Rural population

### Conclusion:
   So by also adding the population data to the covid data, we can do future anaysis on the effect
   of population density on the pandemic. Also adding the Urban population to the Rural population
   ratio gives us more depth in analysing whether the states majorly having rural population or 
   urban population have been affected more.

First importing all the libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

##### Covid data from source 1:

In [3]:
url1 = "https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_India"

In [5]:
response1 = requests.get(url1)
print(response1)

<Response [200]>


In [6]:
soup1 = BeautifulSoup(response1.content, "html.parser")

In [7]:
soup1.title

<title>2020 coronavirus pandemic in India - Wikipedia</title>

In [9]:
tables1 = soup1.find_all("table")

In [10]:
len(tables1)

32

In [11]:
coronatable = tables1[3]

Getting data from the table into the list and converting the string to int

In [12]:
state1 = []
total_cases = []
deaths = []
recoveries = []
active_cases = []
rows = coronatable.find_all("tr")[2:-3]
colname = [total_cases,deaths,recoveries,active_cases]
colnum = [0,1,2,3]

for row in rows:
    col1 = row.find_all("th")
    state1.append(col1[1].text.strip())
    
    col2 = row.find_all("td")
    for i,j in zip(colname,colnum):
        s = col2[j].text.strip()
        s1 = s.replace(",","")
        s2 = s1.split("[", 1)[0]
        s3 = int(s2)
        i.append(s3)

In [13]:
df = pd.DataFrame(list(zip(state1,total_cases,deaths,recoveries,active_cases)), columns=["State","Total_Cases","Deaths","Recoveries","Active_Cases"])
df1 = df.sort_values(['State'],ignore_index=True)
df1.head()

,State,Total_Cases,Deaths,Recoveries,Active_Cases
0,Andaman and Nicobar Islands,33,0,15,18
1,Andhra Pradesh,1259,31,258,970
2,Arunachal Pradesh,1,0,1,0
3,Assam,38,1,27,10
4,Bihar,366,2,64,300


In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   State         36 non-null     object
 1   Total_Cases   36 non-null     int64 
 2   Deaths        36 non-null     int64 
 3   Recoveries    36 non-null     int64 
 4   Active_Cases  36 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 1.5+ KB


##### Population data from source 2:

In [15]:
url2 = "https://en.wikipedia.org/wiki/List_of_states_and_union_territories_of_India_by_population"
response2 = requests.get(url2)
print(response2)

<Response [200]>


In [16]:
soup2 = BeautifulSoup(response2.content, "html.parser")
soup2.title

<title>List of states and union territories of India by population - Wikipedia</title>

In [17]:
tables2 = soup2.find_all("table")
len(tables2)

2

In [18]:
pop_table = tables2[1]

Getting data from the table into the list and converting the string to int

In [19]:
state2 = []
population = []
density = []
rural = []
urban = []
rows = pop_table.find_all("tr")[1:-1]
colname = [population,density,rural,urban]
colnum = [2,7,4,5]
for row in rows:
    col = row.find_all("td")
    state2.append(col[1].text.strip())
    for i,j in zip(colname,colnum):
        s = col[j].text.strip()
        s1 = s.replace(",","")
        s2 = s1.split("[", 1)[0]
        s3 = s2.split("(")[0]
        s4 = s3.split("/")[0]
        s5 = float(s4) #for just one 2.8 float in the table I had to use this else int()would have done it
        s6 = round(s5)
        i.append(s6)

In [20]:
df = pd.DataFrame(list(zip(state2,population,density,rural,urban)), columns=['State','Population','Population_Density','Rural_Population','Urban_Population'])
df2 = df.sort_values('State',ignore_index=True)
df2.head()

,State,Population,Population_Density,Rural_Population,Urban_Population
0,Andaman and Nicobar Islands,380581,46,237093,143488
1,Andhra Pradesh,49577103,303,34966693,14610410
2,Arunachal Pradesh,1383727,17,1066358,317369
3,Assam,31205576,397,26807034,4398542
4,Bihar,104099452,1102,92341436,11758016


In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   State               36 non-null     object
 1   Population          36 non-null     int64 
 2   Population_Density  36 non-null     int64 
 3   Rural_Population    36 non-null     int64 
 4   Urban_Population    36 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 1.5+ KB


##### Merging df1 and df2

In [67]:
df3 = pd.merge(df1, df2, on='State')
df3.head()

,State,Total_Cases,Deaths,Recoveries,Active_Cases,Population,Population_Density,Rural_Population,Urban_Population
0,Andaman and Nicobar Islands,33,0,15,18,380581,46,237093,143488
1,Andhra Pradesh,1259,31,258,970,49577103,303,34966693,14610410
2,Arunachal Pradesh,1,0,1,0,1383727,17,1066358,317369
3,Assam,38,1,27,10,31205576,397,26807034,4398542
4,Bihar,366,2,64,300,104099452,1102,92341436,11758016


##### Adding a column for the ratio of Urban population to the rural population

In [68]:
ratio = (dftemp['Urban_Population'])/(dftemp['Rural_Population'])

In [69]:
ratio_series = pd.Series(ratio,name="Urban_Rural_Ratio")

Concatnating it to df3 to get the final df

In [76]:
df = pd.concat([df3, ratio_series], axis=1)
df

,State,Total_Cases,Deaths,Recoveries,Active_Cases,Population,Population_Density,Rural_Population,Urban_Population,Urban_Rural_Ratio
0,Andaman and Nicobar Islands,33,0,15,18,380581,46,237093,143488,0.605197
1,Andhra Pradesh,1259,31,258,970,49577103,303,34966693,14610410,0.417838
2,Arunachal Pradesh,1,0,1,0,1383727,17,1066358,317369,0.297620
3,Assam,38,1,27,10,31205576,397,26807034,4398542,0.164082
4,Bihar,366,2,64,300,104099452,1102,92341436,11758016,0.127332
5,Chandigarh,56,0,17,39,1055450,9252,28991,1026459,35.406126
6,Chhattisgarh,38,0,34,4,25545198,189,19607961,5937237,0.302797
7,Dadra and Nagar Haveli and Daman and Diu,0,0,0,0,585764,970,243510,342254,1.405503
8,Delhi,3314,54,1078,2182,16787941,11297,419042,16368899,39.062669
9,Goa,7,0,7,0,1458545,394,551731,906814,1.643580


Above is my final dataframe.
Thank You!